#### Week 1. Titanic Classification with DistilBERT + XGBoost

`Titanic: Machine Learning from Disaster`은 머신러닝을 처음 공부했을 때 다들 한번씩은 거쳐갔을 유명한 실습예제 중 하나입니다.

이번 주차 학습 목표는 다음과 같습니다.


1. 온라인 공모전 사이트 `kaggle`에서 데이터를 받고, 모델로 학습한 결과를 csv파일로 업로드하기

2. 자연어처리 모델인 BERT 모델을 Huggingface에서 받아와 분류하기 전 데이터를 숫자 벡터로 Embedding하기

3. Embedding한 숫자 벡터를 머신러닝 알고리즘으로 분류작업 진행하기

Kaggle Titanic Competition 바로가기 : https://www.kaggle.com/competitions/titanic  

### 1. Kaggle에서 데이터 받아오기

In [1]:
## pip install ~ : 패키지 다운로드
## -q : 로그 메세지 출력 X

!pip install -q kaggle

In [2]:
## Kaggle에서 데이터를 받아오기 위해서는 Authenticator Token인 "kaggle.json"이 있어야 합니다.
## 자세한 내용은 영상을 참고하세요

from google.colab import drive
drive.mount("/content/drive")

!mkdir ~/.kaggle
## Drive에 kaggle.json을 업로드한 경로를 적으시면 됩니다. ex) (/content/drive/MyDrive/study_session/kaggle.json)
!cp /content/drive/MyDrive/""your_directory"" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Mounted at /content/drive


In [3]:
## 현재 kaggle에서 진행중인 공모전 리스트를 확인할 수 있습니다.
## 저희는 https://www.kaggle.com/competitions/titanic 예제를 사용하겠습니다.

!kaggle competitions list

ref                                                                                           deadline             category            reward  teamCount  userHasEntered  
--------------------------------------------------------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
https://www.kaggle.com/competitions/asl-fingerspelling                                        2023-08-24 23:59:00  Research          $200,000        541           False  
https://www.kaggle.com/competitions/2023-kaggle-ai-report                                     2023-07-16 23:59:00  Analytics          $70,000        349           False  
https://www.kaggle.com/competitions/icr-identify-age-related-conditions                       2023-08-10 23:59:00  Featured           $60,000       4485           False  
https://www.kaggle.com/competitions/hubmap-hacking-the-human-vasculature                      2023-07-31 23:59:00  Research           $50,000    

In [3]:
## Titanic 공모전 데이터를 코랩으로 받고, zip 압축 해제하기

# 공모전 이름
competition_name = "titanic"

# 공모전 다운로드 to local environment
! kaggle competitions download -c {competition_name}

# {competition_name}이름의 폴더에 zip 파일 압축해제
! unzip {competition_name + ".zip"} -d {competition_name}

# 드라이브 확인을 완료했으므로 드라이브 mount를 해제합니다.
drive.flush_and_unmount()

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 64.6MB/s]
Archive:  titanic.zip
  inflating: titanic/gender_submission.csv  
  inflating: titanic/test.csv        
  inflating: titanic/train.csv       


### 2. 데이터 전처리

In [4]:
import numpy as np
import pandas as pd

train = pd.read_csv('titanic/train.csv')
test = pd.read_csv('titanic/test.csv')

In [10]:
train.tail(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [9]:
train.shape

(891, 12)

In [6]:
from sklearn.model_selection import train_test_split

X = train.drop(columns = ['PassengerId', 'Survived'], axis = 1)
y = train['Survived']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.05, random_state = 77, stratify = y)

In [7]:
## GPU 활성화
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 3. 데이터 임베딩

In [8]:
from typing import List
from tqdm.notebook import tqdm

!pip install -q transformers
from transformers import AutoModel, AutoTokenizer

class Encode_with_BERT:
    def __init__(self):
        ## Huggingface에서 BERT 모델을 받아옵니다.
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        self.model = AutoModel.from_pretrained('bert-base-uncased').to(device)

    ## 입력한 `texts`을 768 길이의 숫자벡터로 특징을 추출합니다.
    def extract(self, texts: List[str]):
        features = np.zeros((len(texts), 768), dtype = np.float16)
        for index, text in enumerate(tqdm(texts)):
            tokenized_text = self.tokenizer(text, return_tensors="pt").to(device)
            model_output = self.model(**tokenized_text)[0].detach().cpu()
            features[index, :] = model_output.numpy().mean(axis=1)

        return features

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 92.1 MB/s eta 0:00:00


### 4. 임베딩한 데이터 머신러닝 알고리즘으로 분류

In [9]:
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

extractor = Encode_with_BERT()
scaler = StandardScaler()
classifier = XGBClassifier(use_label_encoder = False)

## XGBoost 학습
train_labels = [int(_) for _ in y_train.values] ## [0,1,1, ...]
texts = [", ".join(str(_)) for _ in X_train.values] ##[[0,1,male],[2,1,female]...]
train_features = scaler.fit_transform(extractor.extract(texts)) ## to (len(texts), 768) 숫자벡터
classifier.fit(train_features, train_labels) ## XGBoost train

## Prediction with Test Data
answer = [int(_) for _ in y_val.values]
texts = [", ".join(str(_)) for _ in X_val.values]
preds = classifier.predict(scaler.transform(extractor.extract(texts)))

## 모델 예측의 정밀도 측정!
accuracy = accuracy_score(answer, preds)
print(f'\naccuracy : {accuracy*100:.2f}%')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


  0%|          | 0/846 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]


accuracy : 88.89%


### 5. csv로 저장 + kaggle에 submit

In [10]:
import csv

ids = test['PassengerId'].values
texts = [", ".join(str(_)) for _ in test.iloc[:, 1:].values]
preds = classifier.predict(scaler.transform(extractor.extract(texts)))

with open('submission.csv', "w") as to_file:
    csvwriter = csv.writer(to_file, delimiter=",", quotechar='"')
    csvwriter.writerow(["PassengerId", "Survived"])
    for id, pred in zip(ids, preds):
        csvwriter.writerow([id, pred])

  0%|          | 0/418 [00:00<?, ?it/s]

In [11]:
sub = pd.read_csv('submission.csv')
sub.head(5)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
